In [1]:

import requests
from bs4 import BeautifulSoup
import csv
from datetime import datetime, timedelta
import pandas as pd


In [2]:
def fetch_headlines_for_date(date):
    url = f'https://www.wsj.com/news/archive/{date}'
    response = requests.get(url)
    response.raise_for_status()  # Raise an exception for HTTP errors
    
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Initialize a list to hold the headlines
    headlines = []
    
    # New York Times articles typically have titles in h2 or h3 tags
    for headline in soup.find_all(['h2', 'h3']):
        title = headline.get_text(strip=True)
        if title:
            headlines.append(title)
    
    # Exclude the last 26 headlines
    return headlines[:-26]

# Open a CSV file for writing
with open('nytimes_headlines_2018_to_2022.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['date', 'headline']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    # Write the header row
    writer.writeheader()
    
    # Loop through each date from 2020 to 2023
    start_date = datetime(2018, 1, 1)
    end_date = datetime(2022,12, 31)
    current_date = start_date
    
    while current_date <= end_date:
        date_str = current_date.strftime('%Y/%m/%d')
        print(f"Fetching headlines for {date_str}")
        
        try:
            headlines = fetch_headlines_for_date(date_str)
            for headline in headlines:
                writer.writerow({'date': date_str, 'headline': headline})
        except Exception as e:
            print(f"Failed to fetch headlines for {date_str}: {e}")
        
        # Move to the next day
        current_date += timedelta(days=1)

print("Headlines have been saved to nytimes_headlines_2018_to_2022.csv")


Fetching headlines for 2018/01/01
Failed to fetch headlines for 2018/01/01: 403 Client Error: Forbidden for url: https://www.wsj.com/news/archive/2018/01/01
Fetching headlines for 2018/01/02
Failed to fetch headlines for 2018/01/02: 403 Client Error: Forbidden for url: https://www.wsj.com/news/archive/2018/01/02
Fetching headlines for 2018/01/03
Failed to fetch headlines for 2018/01/03: 403 Client Error: Forbidden for url: https://www.wsj.com/news/archive/2018/01/03
Fetching headlines for 2018/01/04
Failed to fetch headlines for 2018/01/04: 403 Client Error: Forbidden for url: https://www.wsj.com/news/archive/2018/01/04
Fetching headlines for 2018/01/05
Failed to fetch headlines for 2018/01/05: 403 Client Error: Forbidden for url: https://www.wsj.com/news/archive/2018/01/05
Fetching headlines for 2018/01/06
Failed to fetch headlines for 2018/01/06: 403 Client Error: Forbidden for url: https://www.wsj.com/news/archive/2018/01/06
Fetching headlines for 2018/01/07
Failed to fetch headline

KeyboardInterrupt: 

In [32]:
from collections import defaultdict

def merge_sentences_by_date(input_csv, output_csv):
    # Initialize a dictionary to hold merged sentences by date
    date_sentences = defaultdict(list)
    
    # Read the input CSV file
    with open(input_csv, 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            date = row['date']
            sentence = row['headline']
            date_sentences[date].append(sentence)
    
    # Write the merged results to the output CSV file
    with open(output_csv, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['date', 'headline']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        # Write the header row
        writer.writeheader()
        
        # Write the merged sentences
        for date, sentences in date_sentences.items():
            merged_sentences = ' '.join(sentences)
            writer.writerow({'date': date, 'headline': merged_sentences})

# Example usage
input_csv = 'nytimes_headlines_2018_to_2022.csv'
output_csv = 'nytimes_headlines_2018_to_2022.csv'
merge_sentences_by_date(input_csv, output_csv)

merge_sentences_by_date(input_csv, output_csv)

print(f"Merged headlines have been saved to {output_csv}")

Merged headlines have been saved to nytimes_headlines_2018_to_2022.csv


In [33]:
pd.read_csv(input_csv)

,date,headline
0,2018/01/01,Top News Kim Jong-un Offers North Korea’s Hand...
1,2018/01/02,"Top News The Trump Effect: Business, Anticipat..."
2,2018/01/03,Top News Hard-Liners and Reformers Tapped Iran...
3,2018/01/04,Top News Obstruction Inquiry Shows Trump’s Str...
4,2018/01/05,Top News Obstruction Inquiry Shows Trump’s Str...
...,...,...
1819,2022/12/27,Top News George Santos Admits to Lying About C...
1820,2022/12/28,Top News ‘Tragic Battle’: On the Front Lines o...
1821,2022/12/29,Top News U.S. Scrambles to Stop Iran From Prov...
1822,2022/12/30,Top News Russian Missile Barrage Staggers Ukra...


In [34]:
pd.read_csv('nytimes_headlines_2018_to_2022.csv')

,date,headline
0,2018/01/01,Top News Kim Jong-un Offers North Korea’s Hand...
1,2018/01/02,"Top News The Trump Effect: Business, Anticipat..."
2,2018/01/03,Top News Hard-Liners and Reformers Tapped Iran...
3,2018/01/04,Top News Obstruction Inquiry Shows Trump’s Str...
4,2018/01/05,Top News Obstruction Inquiry Shows Trump’s Str...
...,...,...
1819,2022/12/27,Top News George Santos Admits to Lying About C...
1820,2022/12/28,Top News ‘Tragic Battle’: On the Front Lines o...
1821,2022/12/29,Top News U.S. Scrambles to Stop Iran From Prov...
1822,2022/12/30,Top News Russian Missile Barrage Staggers Ukra...


In [36]:
def adjust_weekend_headlines(input_csv, output_csv):
    # Read the CSV file into a DataFrame
    data = pd.read_csv(input_csv, parse_dates=['date'])

    # Ensure the date column is in datetime format
    data['date'] = pd.to_datetime(data['date'])

    # Sort the DataFrame by date
    data = data.sort_values(by='date')

    # Initialize a dictionary to collect headlines for each Monday
    monday_headlines = {}

    # Iterate over the rows to collect weekend headlines and associate them with the following Monday
    for index, row in data.iterrows():
        date = row['date']
        weekday = date.weekday()
        if weekday == 5:  # Saturday
            next_monday = date + timedelta(days=2)
        elif weekday == 6:  # Sunday
            next_monday = date + timedelta(days=1)
        else:
            continue

        if next_monday not in monday_headlines:
            monday_headlines[next_monday] = {'headline': ''}

        monday_headlines[next_monday]['headline'] += ' ' + row['headline']

    # Remove Saturday and Sunday rows
    weekday_data = data[~data['date'].dt.weekday.isin([5, 6])]

    # Append weekend headlines to the following Monday
    for next_monday, values in monday_headlines.items():
        if next_monday in weekday_data['date'].values:
            weekday_data.loc[weekday_data['date'] == next_monday, 'headline'] += values['headline']
        else:
            new_row = pd.DataFrame({'date': [next_monday], 'headline': [values['headline']]})
            weekday_data = pd.concat([weekday_data, new_row], ignore_index=True)

    # Sort the DataFrame by date again after modification
    weekday_data = weekday_data.sort_values(by='date').reset_index(drop=True)

    # Write the updated DataFrame to the output CSV file
    weekday_data.to_csv(output_csv, index=False, encoding='utf-8')

In [37]:
output_csv1 = 'final_headlines_weekday.csv'
input_csv1 = 'nytimes_headlines_2018_to_2022.csv'
adjust_weekend_headlines(input_csv1, output_csv1)

print(f"Weekend headlines have been adjusted and saved to {output_csv1}")

Weekend headlines have been adjusted and saved to final_headlines_weekday.csv


In [38]:
pd.read_csv(output_csv1)

,date,headline
0,2018-01-01,Top News Kim Jong-un Offers North Korea’s Hand...
1,2018-01-02,"Top News The Trump Effect: Business, Anticipat..."
2,2018-01-03,Top News Hard-Liners and Reformers Tapped Iran...
3,2018-01-04,Top News Obstruction Inquiry Shows Trump’s Str...
4,2018-01-05,Top News Obstruction Inquiry Shows Trump’s Str...
...,...,...
1300,2022-12-27,Top News George Santos Admits to Lying About C...
1301,2022-12-28,Top News ‘Tragic Battle’: On the Front Lines o...
1302,2022-12-29,Top News U.S. Scrambles to Stop Iran From Prov...
1303,2022-12-30,Top News Russian Missile Barrage Staggers Ukra...
